In [1]:
import model_lib
import numpy as np
import warnings
warnings.filterwarnings('ignore', '.*output shape of zoom.*')
import pickle
import importlib
importlib.reload(model_lib)
import os
import time
from PIL import Image

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


In [2]:
# config to train
# TODO: check Config is correct
class ProposalConfig():
    NAME = "InSegm"
    GPU_COUNT = 1
    # online training
    IMAGES_PER_GPU = 1
    STEPS_PER_EPOCH = 100
    # not going to use these
    N_DISTORTIONS = 0
    MAX_DISTORTION = 0.3
    MIN_DISTORTION = -0.1
    
    VALIDATION_STEPS = 20
    # including gt
    NUM_CLASSES = 81
    # only flips
    IMAGE_AUGMENT = True

    MEAN_PIXEL = np.array([123.7, 116.8, 103.9],dtype=np.float32)
    MAX_GT_INSTANCES = 100
    DETECTION_MAX_INSTANCES = 100
    DETECTION_MIN_CONFIDENCE = 0.7
    CLASS_NAMES = [
        'BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
        'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
        'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
        'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella',
        'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard',
        'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
        'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork',
        'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
        'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
        'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv',
        'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
        'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
        'scissors', 'teddy bear', 'hair drier', 'toothbrush'
    ]
    DETECTION_NMS_THRESHOLD = 0.3
    LEARNING_RATE = 0.05
    LEARNING_MOMENTUM = 0.9
    WEIGHT_DECAY = 0.0001
    WIDTH = 224
    HEIGHT = 224
    MASK_SHAPE = (64,64)
    GRID_WIDTH = 16
    GRID_HEIGHT = 16
    CLUE_SHAPE = (20,20)
    GRID_SHAPE = (GRID_WIDTH, GRID_HEIGHT)
    GRID_RESOLUTION = (1, 1)
    IS_PADDED = True
    MASK_THRESOLD = 0.7
    def __init__(self):
        self.BATCH_SIZE = self.IMAGES_PER_GPU * self.GPU_COUNT
        self.IMAGE_SHAPE = (self.WIDTH, self.HEIGHT,3)
        self.MAX_BATCH_SIZE = self.BATCH_SIZE*32

    def display(self):
        """Display Configuration values."""
        print("\nConfigurations:")
        for a in dir(self):
            if not a.startswith("__") and not callable(getattr(self, a)):
                print("{:30} {}".format(a, getattr(self, a)))
        print("\n")

In [3]:
root_dir = "/media/data/nishanth/aravind/"
config = ProposalConfig()
model_dir = "./models/"
train_dataset,val_dataset = None,None
train_pickle = root_dir+"val_cid.pickle"
val_pickle = root_dir+"val_cid.pickle"
class ClassInstancesDataset():
    def __init__(self):
        self.class_wise_instance_info = [[] for i in range(81)]
        self.instance_info = []
import time
start = time.time()
with open(train_pickle,"rb") as train_ann:
    train_cid = pickle.load(train_ann)
end = time.time()
print(end-start)
#     train_dataset = model_lib.CocoDataset(train_cid,config)
with open(val_pickle,"rb") as val_ann:
    val_cid = pickle.load(val_ann)
#     val_dataset = model_lib.CocoDataset(val_cid,config)
train_loader = model_lib.get_loader(train_cid,config,"cw_ins")
val_loader = model_lib.get_loader(val_cid,config,"ins")

10.518236875534058


In [ ]:
import torch
import torch.nn.functional as F
net = model_lib.SimpleHGModel()
net.load_state_dict(torch.load(model_dir+"model_6_7400.pt"))
net = net.cuda()
with torch.no_grad():
    for i,data in enumerate(val_loader):
        batch_images,batch_impulses,batch_gt_responses,batch_class_ids = data
        a,b,c= batch_images.numpy(), batch_impulses.numpy(),batch_gt_responses.numpy()
        a = np.moveaxis(a,1,-1)
        b = np.moveaxis(b,1,-1)
        c = np.moveaxis(c,1,-1)
        Image.fromarray((a[0]*128 + config.MEAN_PIXEL).astype(np.uint8),"RGB").show()
        Image.fromarray((b[0][:,:,0]*128).astype(np.uint8),"L").show()
        Image.fromarray((c[0][:,:,0]*128).astype(np.uint8),"L").show()
        batch_images,batch_impulses,batch_gt_responses,batch_class_ids = batch_images.cuda(),batch_impulses.cuda(),batch_gt_responses.cuda(),batch_class_ids.cuda()
        pred_class = net([batch_images,batch_impulses])
#         print(pred_class)
        pred_class = F.softmax(pred_class,dim=-1).squeeze()
        maxs, indices = torch.topk(pred_class,5,-1)
        print(maxs.shape,indices.shape)
        for i in range(5):
            print(maxs[i],indices[i],config.CLASS_NAMES[int(indices[i])])
#         print(batch_class_ids)
#         print(indices)
        print("gt_class:",config.CLASS_NAMES[int(batch_class_ids[0])])
        print("pred_class: ",pred_class[int(batch_class_ids[0])])
#         print(config.CLASS_NAMES[int(indices[0])])
        input()

torch.Size([5]) torch.Size([5])
tensor(0.1134, device='cuda:0') tensor(1, device='cuda:0') person
tensor(1.00000e-02 *
       9.8686, device='cuda:0') tensor(29, device='cuda:0') suitcase
tensor(1.00000e-02 *
       9.4465, device='cuda:0') tensor(57, device='cuda:0') chair
tensor(1.00000e-02 *
       5.7191, device='cuda:0') tensor(60, device='cuda:0') bed
tensor(1.00000e-02 *
       4.9479, device='cuda:0') tensor(59, device='cuda:0') potted plant
gt_class: person
pred_class:  tensor(0.1134, device='cuda:0')

torch.Size([5]) torch.Size([5])
tensor(0.1070, device='cuda:0') tensor(57, device='cuda:0') chair
tensor(0.1056, device='cuda:0') tensor(56, device='cuda:0') cake
tensor(1.00000e-02 *
       8.3686, device='cuda:0') tensor(29, device='cuda:0') suitcase
tensor(1.00000e-02 *
       7.6748, device='cuda:0') tensor(59, device='cuda:0') potted plant
tensor(1.00000e-02 *
       6.9690, device='cuda:0') tensor(1, device='cuda:0') person
gt_class: bowl
pred_class:  tensor(1.00000e-02 *



torch.Size([5]) torch.Size([5])
tensor(0.1190, device='cuda:0') tensor(58, device='cuda:0') couch
tensor(1.00000e-02 *
       9.0485, device='cuda:0') tensor(1, device='cuda:0') person
tensor(1.00000e-02 *
       7.4226, device='cuda:0') tensor(57, device='cuda:0') chair
tensor(1.00000e-02 *
       6.9463, device='cuda:0') tensor(60, device='cuda:0') bed
tensor(1.00000e-02 *
       5.5157, device='cuda:0') tensor(14, device='cuda:0') bench
gt_class: BG
pred_class:  tensor(1.00000e-08 *
       5.8537, device='cuda:0')

torch.Size([5]) torch.Size([5])
tensor(0.1259, device='cuda:0') tensor(29, device='cuda:0') suitcase
tensor(0.1031, device='cuda:0') tensor(1, device='cuda:0') person
tensor(1.00000e-02 *
       7.3323, device='cuda:0') tensor(57, device='cuda:0') chair
tensor(1.00000e-02 *
       5.8844, device='cuda:0') tensor(25, device='cuda:0') backpack
tensor(1.00000e-02 *
       5.8339, device='cuda:0') tensor(27, device='cuda:0') handbag
gt_class: person
pred_class:  tensor(0.103


torch.Size([5]) torch.Size([5])
tensor(0.1540, device='cuda:0') tensor(59, device='cuda:0') potted plant
tensor(0.1187, device='cuda:0') tensor(29, device='cuda:0') suitcase
tensor(1.00000e-02 *
       8.6416, device='cuda:0') tensor(8, device='cuda:0') truck
tensor(1.00000e-02 *
       6.2184, device='cuda:0') tensor(9, device='cuda:0') boat
tensor(1.00000e-02 *
       4.9216, device='cuda:0') tensor(56, device='cuda:0') cake
gt_class: surfboard
pred_class:  tensor(1.00000e-03 *
       1.4411, device='cuda:0')

torch.Size([5]) torch.Size([5])
tensor(0.2003, device='cuda:0') tensor(1, device='cuda:0') person
tensor(0.1282, device='cuda:0') tensor(57, device='cuda:0') chair
tensor(1.00000e-02 *
       5.5212, device='cuda:0') tensor(29, device='cuda:0') suitcase
tensor(1.00000e-02 *
       4.5382, device='cuda:0') tensor(25, device='cuda:0') backpack
tensor(1.00000e-02 *
       4.2633, device='cuda:0') tensor(76, device='cuda:0') vase
gt_class: person
pred_class:  tensor(0.2003, device

In [32]:
import random
import time
np.set_printoptions(threshold=np.nan)
counts = [0 for i in range(81)]
for i,g in enumerate(train_loader):
    print("sample:" + str(i))
    # start = time.time()
    batch_images,batch_impulses,batch_gt_responses,batch_class_ids = g
#     print(batch_class_ids)
    for i in batch_class_ids:
        counts[int(i.item())] += 1
print(counts)
#     batch_images = np.moveaxis(batch_images.numpy(),1,-1)
#     print(batch_images.shape)
#     batch_impulses = batch_impulses.squeeze().numpy()*128
#     batch_gt_responses = batch_gt_responses.squeeze().numpy()*128
#     batch_class_ids = batch_class_ids.numpy()
#     # s =(time.time()-start)
#     # print(s)
#     batch_images[0] *= 128
#     batch_images[0] += config.MEAN_PIXEL
#     img = Image.fromarray((batch_images[0]).astype("uint8"),"RGB")
#     img.show()
#     print((np.sum(batch_gt_responses)//128)**0.5)
#     mask = Image.fromarray(((batch_gt_responses)).astype("uint8"),"L")
#     mask.show()
#     print(config.CLASS_NAMES[batch_class_ids[0]])
#     impulse = Image.fromarray(((batch_impulses)).astype("uint8"),"L")
#     impulse.show()
#     input()

sample:0
sample:1
sample:2
sample:3
sample:4
sample:5
sample:6
sample:7
sample:8
sample:9
sample:10
sample:11
sample:12
sample:13
sample:14
sample:15
sample:16
sample:17
sample:18
sample:19
sample:20
sample:21
sample:22
sample:23
sample:24
sample:25
sample:26
sample:27
sample:28
sample:29
sample:30
sample:31
sample:32
sample:33
sample:34
sample:35
sample:36
sample:37
sample:38
sample:39
sample:40
sample:41
sample:42
sample:43
sample:44
sample:45
sample:46
sample:47
sample:48
sample:49
sample:50
sample:51
sample:52
sample:53
sample:54
sample:55
sample:56
sample:57
sample:58
sample:59
sample:60
sample:61
sample:62
sample:63
sample:64
sample:65
sample:66
sample:67
sample:68
sample:69
sample:70
sample:71
sample:72
sample:73
sample:74
sample:75
sample:76
sample:77
sample:78
sample:79
sample:80
sample:81
sample:82
sample:83
sample:84
sample:85
sample:86
sample:87
sample:88
sample:89
sample:90
sample:91
sample:92
sample:93
sample:94
sample:95
sample:96
sample:97
sample:98
sample:99
sample:100

sample:758
sample:759
sample:760
sample:761
sample:762
sample:763
sample:764
sample:765
sample:766
sample:767
sample:768
sample:769
sample:770
sample:771
sample:772
sample:773
sample:774
sample:775
sample:776
sample:777
sample:778
sample:779
sample:780
sample:781
sample:782
sample:783
sample:784
sample:785
sample:786
sample:787
sample:788
sample:789
sample:790
sample:791
sample:792
sample:793
sample:794
sample:795
sample:796
sample:797
sample:798
sample:799
sample:800
sample:801
sample:802
sample:803
sample:804
sample:805
sample:806
sample:807
sample:808
sample:809
sample:810
sample:811
sample:812
sample:813
sample:814
sample:815
sample:816
sample:817
sample:818
sample:819
sample:820
sample:821
sample:822
sample:823
sample:824
sample:825
sample:826
sample:827
sample:828
sample:829
sample:830
sample:831
sample:832
sample:833
sample:834
sample:835
sample:836
sample:837
sample:838
sample:839
sample:840
sample:841
sample:842
sample:843
sample:844
sample:845
sample:846
sample:847
sample:848

In [30]:
[len(c) for c in train_cid.class_wise_instance_info]

[4952,
 10777,
 314,
 1919,
 367,
 146,
 283,
 222,
 414,
 424,
 634,
 101,
 75,
 60,
 411,
 427,
 202,
 218,
 272,
 354,
 374,
 252,
 72,
 266,
 252,
 377,
 417,
 541,
 254,
 305,
 119,
 243,
 69,
 263,
 336,
 148,
 148,
 180,
 267,
 229,
 1015,
 353,
 895,
 217,
 325,
 253,
 624,
 379,
 239,
 177,
 285,
 312,
 369,
 127,
 284,
 334,
 310,
 1771,
 263,
 342,
 166,
 703,
 186,
 289,
 231,
 106,
 296,
 155,
 262,
 55,
 143,
 12,
 231,
 127,
 1129,
 269,
 274,
 40,
 204,
 13,
 284]